In [ ]:
# !pip install boto3
# !pip install python-dotenv


In [1]:
import importlib

import run_services as rn

# flights
import flights.aggregator as ag
from datetime import date, timedelta

# video
import media.video_generator as vg

# caption
import content.caption_builder as cb

# instagram
from pathlib import Path
import os
from instagram.ig_client import InstagramClient

# review
import uuid
from pathlib import Path
import review.telegram_review as tr

import affiliates.affiliates as af


In [ ]:
proc = rn.start_services()

In [ ]:
# rn.stop_services(proc)

## FLIGHTS

In [2]:
import pandas as pd

In [3]:
importlib.reload(ag)


<module 'flights.aggregator' from 'C:\\Users\\Macia\\Desktop\\Jupyter Notebooks\\EscapadasMallorca\\flights\\aggregator.py'>

In [42]:
start = date.today()
end = start + timedelta(days=15)

flights = ag.get_flights_in_period(start, end)
best_by_cat = ag.get_best_by_category_scored(flights, min_discount_pct=40.0)
# best_by_cat = ag.get_best_by_category_scored(flights[25:35])

for item in best_by_cat:
    f = item["flight"]
    cat = item["category"]
    score = item["score"]
    print(cat["label"], f"{f.origin}→{f.destination}", f.price, f.price_per_km, score)

🔎 Buscando vuelos entre 2025-12-06 y 2025-12-21...
🗓  Buscando vuelos 2025-12-11 → 2025-12-14...
🗓  Buscando vuelos 2025-12-11 → 2025-12-15...
🗓  Buscando vuelos 2025-12-12 → 2025-12-14...
🗓  Buscando vuelos 2025-12-12 → 2025-12-15...
🗓  Buscando vuelos 2025-12-18 → 2025-12-21...
🗓  Buscando vuelos 2025-12-19 → 2025-12-21...
✅ Encontrados 114 vuelos en total.
🔥 Ultra Chollo PMI→VIE 42.269999999999996 0.02899225913475379 10.830385940368837
🍝 Escapada Gastronómica PMI→ZAZ 111.0 0.27975905436398923 1.5119549549549547


In [43]:
main_item = ag.choose_main_candidate_prob(best_by_cat)

# main_item = best_by_cat[1]

if main_item is None:
    raise ValueError("No hay candidatos para hoy")

main_flight = main_item["flight"]
main_category_code = main_item["category"]["code"]

main_flight

Flight(origin='PMI', destination='ZAZ', price=111.0, start_date='2025-12-18T15:05:00.000Z', end_date='2025-12-21T08:40:00.000Z', airline='VY', link='https://www.kiwi.com/deep?affilid=travelpayoutsapi&currency=EUR&flightsId=13a2102c4fd80000d3e3e0cd_0%7C102c13a24fdb0000aefcfdda_0&from=PMI&lang=en&passengers=1&to=ZAZ&booking_token=HVvWPOAARRY9MAwrdJyc9nVr14k6y2x2ita03gHXDWaecVAkcIxvO7b8BKrqaPVaz9m2PC-lHcPcffRQ6d8JhmTzXFLLHtx72i0fKkUt3qbo3WidBeWkRaR07KcMGn9YPr_A2JQjkQ8A9AYL1WttAOvLJ5C_pntM28fvFh_iSsE52WqaUCPLvt4PyMZMNXYlV2-mcZ8PzLDkWn1cWQ4Of5dCY6IGlmrPe9n_yC1m7HwernhEn78hHkFHJYWLERkNDiTVNLPmOrUwCyTGj2SGRtkHoTLLf_zebKGCPZiFicZ5bI4AYHPjUqWTnoTWWZ20BsZBBtoD1p5QLAoKT3TIo1nJ8uZ9b353V5uPtL3QHqXjIbsKxjE8q-vUHzh_kX10LG-C2fr1ifiIt9ov03oeflfSb_54K0qDLewnJqRKxB3ZTfX-q3BDXtOtTsTQaCOmGYSkUW3y05B2BLgnDOnQGbPFjEGe6iIug6UvVPKiKPgfQNhGJQiuEy3m9ESD8qiu5aUBSD-wqB8S_AAZBrk8F4IwbC9h57sgq6F5KVWBJ2OUdRBJqEOEFeHPEWae0JreL7UfjhSQAhd-rcuMvPGPSmI9O0rtB8KIHVOB1DDS6zteEeXGeASInWJGEDo5awqcfvOFK-lxVyfDEVqH10N7v0ISjKhF2o

In [44]:
NGROK_BASE_URL = "https://karie-apocatastatic-maddox.ngrok-free.dev" 
BASE_DIR = "/media/videos/reel.mp4"
No_cache = "?cache_bust=1"

video_url = NGROK_BASE_URL + BASE_DIR + No_cache
video_url

'https://karie-apocatastatic-maddox.ngrok-free.dev/media/videos/reel.mp4?cache_bust=1'

In [45]:
from web.exporter import update_flights_json

import affiliates.affiliates as af
importlib.reload(af)

import web.exporter as ex
import web.uploader as up

importlib.reload(ex)
importlib.reload(up)




flight = main_item["flight"]   # tu Flight(...)
affiliate_url = af.build_affiliate_url_for_flight(flight)

data = ex.update_flights_json(
    main_item=main_item,
    json_path="local_copy.json",  # opcional
    market="PMI",
    reel_url=video_url,
    affiliate_url=affiliate_url,
    max_entries=5,
)

up.upload_flights_json(data, key="pmi/flights_of_the_day.json")


JSON subido a: https://escapadasgo-public.s3.eu-north-1.amazonaws.com/pmi/flights_of_the_day.json


In [46]:
affiliate_url

'https://c111.travelpayouts.com/click?shmarker=519984&promo_id=3791&source_type=customlink&type=click&custom_url=https%253A%252F%252Fwww.kiwi.com%252Fdeep%253Ffrom%253DPMI%2526to%253DZAZ%2526departure%253D2025-12-18%2526return%253D2025-12-21'

In [47]:

# 7) Actualizar JSON de la web
update_flights_json(
    main_item=main_item,
    json_path=Path("web/mallorca/flights_of_the_day.json"),
    market="PMI",
    reel_url=video_url,
    affiliate_url=affiliate_url,         # aquí puedes meter tu link de Skyscanner/Tequila
    max_entries=5,
)

{'market': 'PMI',
 'updated_at': '2025-12-06T20:58:00Z',
 'today_id': '2025-12-06_pmi_zaz',
 'flights': [{'id': '2025-12-06_pmi_zaz',
   'date_featured': '2025-12-06',
   'market': 'PMI',
   'category_code': 'gastronomica',
   'category_label': '🍝 Escapada Gastronómica',
   'origin_iata': 'PMI',
   'origin_city': 'Mallorca',
   'destination_iata': 'ZAZ',
   'destination_city': 'Zaragoza',
   'start_date': '2025-12-18T15:05:00.000Z',
   'end_date': '2025-12-21T08:40:00.000Z',
   'price_eur': 111.0,
   'price_per_km': 0.27975905436398923,
   'distance_km': 396.77,
   'score': 1.5119549549549547,
   'airline': 'VY',
   'booking_url': 'https://www.kiwi.com/deep?affilid=travelpayoutsapi&currency=EUR&flightsId=13a2102c4fd80000d3e3e0cd_0%7C102c13a24fdb0000aefcfdda_0&from=PMI&lang=en&passengers=1&to=ZAZ&booking_token=HVvWPOAARRY9MAwrdJyc9nVr14k6y2x2ita03gHXDWaecVAkcIxvO7b8BKrqaPVaz9m2PC-lHcPcffRQ6d8JhmTzXFLLHtx72i0fKkUt3qbo3WidBeWkRaR07KcMGn9YPr_A2JQjkQ8A9AYL1WttAOvLJ5C_pntM28fvFh_iSsE52WqaUCP

## VIDEO

In [48]:
main_flight

Flight(origin='PMI', destination='ZAZ', price=111.0, start_date='2025-12-18T15:05:00.000Z', end_date='2025-12-21T08:40:00.000Z', airline='VY', link='https://www.kiwi.com/deep?affilid=travelpayoutsapi&currency=EUR&flightsId=13a2102c4fd80000d3e3e0cd_0%7C102c13a24fdb0000aefcfdda_0&from=PMI&lang=en&passengers=1&to=ZAZ&booking_token=HVvWPOAARRY9MAwrdJyc9nVr14k6y2x2ita03gHXDWaecVAkcIxvO7b8BKrqaPVaz9m2PC-lHcPcffRQ6d8JhmTzXFLLHtx72i0fKkUt3qbo3WidBeWkRaR07KcMGn9YPr_A2JQjkQ8A9AYL1WttAOvLJ5C_pntM28fvFh_iSsE52WqaUCPLvt4PyMZMNXYlV2-mcZ8PzLDkWn1cWQ4Of5dCY6IGlmrPe9n_yC1m7HwernhEn78hHkFHJYWLERkNDiTVNLPmOrUwCyTGj2SGRtkHoTLLf_zebKGCPZiFicZ5bI4AYHPjUqWTnoTWWZ20BsZBBtoD1p5QLAoKT3TIo1nJ8uZ9b353V5uPtL3QHqXjIbsKxjE8q-vUHzh_kX10LG-C2fr1ifiIt9ov03oeflfSb_54K0qDLewnJqRKxB3ZTfX-q3BDXtOtTsTQaCOmGYSkUW3y05B2BLgnDOnQGbPFjEGe6iIug6UvVPKiKPgfQNhGJQiuEy3m9ESD8qiu5aUBSD-wqB8S_AAZBrk8F4IwbC9h57sgq6F5KVWBJ2OUdRBJqEOEFeHPEWae0JreL7UfjhSQAhd-rcuMvPGPSmI9O0rtB8KIHVOB1DDS6zteEeXGeASInWJGEDo5awqcfvOFK-lxVyfDEVqH10N7v0ISjKhF2o

In [49]:
# from flights.base import Flight   # o desde donde lo tengas

# main_flight = Flight(
#     origin='PMI',
#     destination='LPA',
#     price=74.23,
#     start_date='2025-12-11 09:20:00',
#     end_date='2025-12-15 13:30:00',
#     airline='Ryanair',
#     link='https://www.ryanair.com/es/es/trip/flights/select?...',
#     distance_km=837.070500454846,
#     price_per_km=0.0886783131882739,
#     route_typical_price=126.60,
#     discount_pct=41.4,
#     category_code='cultural',
#     category_label='Escapada Cultural'
# )

In [50]:
importlib.reload(vg)

logo_path = "media/images/EscapGo_circ_logo_transparent.png"   # circular, transparente
out_path = "media/videos/reel.mp4"

vg.create_reel_for_flight(
    main_flight,               # tu objeto Flight o dict
    out_mp4_path=out_path,
    logo_path=logo_path,
    duration=4.0,
)

Moviepy - Building video media/videos/reel.mp4.
Moviepy - Writing video media/videos/reel.mp4



Moviepy - Done !
Moviepy - video ready media/videos/reel.mp4


'media/videos/reel.mp4'

## CAPTION

In [51]:
importlib.reload(cb)
caption_text = cb.build_caption_for_flight(
    main_flight,
    category_code=main_category_code,  # "cultural", "romantica", etc.
    tone="emocional",
)

caption_text

'Descubre Zaragoza con un vuelo increíble desde Mallorca.\n\nTe cuento fechas y un plan gastronómico que no olvidarás. Este vuelo es aprox. un 41% más barato que lo habitual.\n\n📅 18 de diciembre: salida desde Mallorca (PMI) a las 15:05\n📅 21 de diciembre: regreso desde Zaragoza (ZAZ) a las 08:40\n💸 111€ ida y vuelta\n\n🇪🇸 Día 1, Tapas en el casco antiguo:\n- Prueba el ternasco asado\n- Pasea por la Plaza del Pilar\n- Visita el Mercado Central\n🇪🇸 Día 2, Cultura y sabores:\n- Tour por bodegas locales\n- Degusta el bacalao ajoarriero\n- Relájate en la ribera del Ebro\n\nZaragoza ofrece una mezcla única de historia y sabores que cautivan todos los sentidos. Su ambiente vibrante y su exquisita gastronomía harán que esta escapada sea inolvidable.\n\n¿Quién se apunta a esta aventura culinaria? Etiquétal@ y organizad la escapada.\n\n#viajar #vuelosbaratos #escapadas #mallorca #gastronomia #Zaragoza #viajes #foodie #turismo'

## REVIEW

In [52]:
# Después de elegir tu "mejor vuelo", generar caption y vídeo:
job_id = str(uuid.uuid4())

local_video_path = Path("media/videos/reel.mp4")

review_candidates = tr.to_review_candidates(best_by_cat)

tr.register_job(
    job_id=job_id,
    flight=best_by_cat[0]['flight'],
    caption=caption_text,
    video_path=local_video_path,
    candidates=review_candidates,  # si quieres pasarle más opciones
)

tr.send_review_candidate(job_id)
print(f"Enviado a Telegram para revisión. job_id={job_id}")

Enviado a Telegram para revisión. job_id=7041a709-1363-4528-926e-577ce0f93f41


## INSTAGRAM

In [ ]:
video_url

In [ ]:
caption_text = "Test Reel desde @escapadas_mallorca 🚀"

# 4) Cliente IG
ig = InstagramClient()

# 5) Crear contenedor
creation_id = ig.create_reel_container(video_url=video_url, caption=caption_text)

# 6) Esperar a que esté listo
if ig.wait_until_ready(creation_id):
    # 7) Publicar
    reel_id = ig.publish_reel(creation_id)
    permalink = ig.get_media_permalink(reel_id)
    print("✅ Reel publicado:")
    print("ID:", reel_id)
    print("Permalink:", permalink)
else:
    print("❌ No se pudo procesar el vídeo, no se publica.")

In [ ]:
import pandas as pd
pd.DataFrame(flights).sort_values(by='price')

In [ ]:
# main.py

import time
from datetime import date

from config.settings import SETTINGS
from flights.aggregator import get_best_daily_flight
from content.caption_builder import build_caption
from media.image_picker import pick_image_for_destination
from media.video_generator import generate_reel_video
from storage.uploader import upload_video_and_get_url
from instagram.publish import publish_reel
from review.telegram_bot import send_review_message, wait_for_user_decision

In [ ]:
def daily_workflow():
    print("🚀 Iniciando flujo diario...")

    # 1) Obtener mejor oferta del día (de cualquier API disponible)
    print("🔍 Buscando mejor vuelo del día...")
    flight = get_best_daily_flight(date.today())
    if not flight:
        print("❌ No se encontró ningún vuelo.")
        return

    # 2) Generar caption con OpenAI (JSON estrictamente controlado)
    print("✍️ Generando caption...")
    caption_json = build_caption(flight)
    caption_text = caption_json_to_string(caption_json)  # si necesitas convertir

    # 3) Seleccionar imagen asociada al destino
    print("🖼 Seleccionando imagen...")
    image_path = pick_image_for_destination(flight.destination)

    # 4) Generar vídeo estilo Reel (6s, zoom, texto)
    print("🎬 Generando vídeo...")
    video_local_path = generate_reel_video(image_path, flight)

    # 5) Enviar todo a Telegram para revisión manual
    print("📲 Enviando a Telegram para revisión...")
    send_review_message(flight, caption_text, video_local_path)

    # 6) Esperar la decisión humana (bloqueante o async)
    print("⌛ Esperando decisión del usuario...")
    decision = wait_for_user_decision()

    if decision == "approve":
        print("👍 Usuario aprobó. Publicando en Instagram...")

        # 7) Subir vídeo a una URL pública
        print("🌐 Subiendo vídeo...")
        video_url = upload_video_and_get_url(video_local_path)

        # 8) Publicar en Instagram
        print("📤 Publicando Reel...")
        permalink = publish_reel(video_url, caption_text)

        print("🎉 Publicado con éxito:", permalink)
        return permalink

    elif decision == "reject":
        print("❌ Usuario rechazó el vuelo. Buscando otra oferta...")
        # Recursivo o bucle para repetir
        return daily_workflow()

    else:
        print("⚠️ Decisión desconocida:", decision)


def caption_json_to_string(caption_json: dict) -> str:
    """
    Convierte el JSON de OpenAI al caption final formateado.
    """
    return (
        f"{caption_json['hook']}\n\n"
        f"{caption_json['summary_line']}\n\n"
        f"{caption_json['dates_block']}\n\n"
        f"{caption_json['itinerary_block']}\n\n"
        f"{caption_json['tips_block']}\n\n"
        f"{caption_json['cta_block']}\n\n"
        f"{caption_json['hashtags']}"
    )

In [ ]:

if __name__ == "__main__":
    daily_workflow()

In [ ]:
# import subprocess
# import sys
# import os
# import time
# import signal

# BASE_DIR = r"C:\Users\Macia\Desktop\Jupyter Notebooks\EscapadasMallorca"

# def start_services():
#     cmd = [sys.executable, "run_services.py"]

#     # Lanzamos el proceso, pero sin bloquear el hilo principal
#     proc = subprocess.Popen(
#         cmd,
#         cwd=BASE_DIR,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.PIPE,
#         text=True
#     )
#     print(f"Servicios iniciados (PID {proc.pid})")
#     return proc

# def stop_services(proc):
#     print(f"Deteniendo servicios (PID {proc.pid})...")

#     # En Windows, Popen.terminate() envía una señal de terminación compatible
#     proc.terminate()

#     try:
#         proc.wait(timeout=10)
#         print("Servicios detenidos correctamente.")
#     except subprocess.TimeoutExpired:
#         print("Forzando cierre...")
#         proc.kill()

# if __name__ == "__main__":
#     proc = start_services()

#     print("\nPresiona ENTER para detener todo.\n")
#     input()

#     stop_services(proc)